In [6]:
from pathlib import Path
import json
import pandas as pd

# Top transaction Data District

In [7]:
def Top_Transcations_Distrit():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})



    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            amount_value = k['metric']['amount']
            amount_formatted = f'{amount_value: .2f}'
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        disrict_name = k['entityName'],
                        type = k['metric']['type'],
                        count = k['metric']['count'],
                        amount = amount_formatted,
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)
    
    return lis2





In [8]:
TopDistrictName =pd.DataFrame(Top_Transcations_Distrit())
TopDistrictName['year'] = TopDistrictName['year'].astype(int)
TopDistrictName['amount'] = TopDistrictName['amount'].astype(float)
TopDistrictName['timestamp'] = pd.to_datetime(TopDistrictName['timestamp'], unit='ms')
TopDistrictName['timestamp'] = TopDistrictName['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
TopDistrictName.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6808 entries, 0 to 6807
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         6808 non-null   object 
 1   year          6808 non-null   int32  
 2   quarter       6808 non-null   object 
 3   disrict_name  6808 non-null   object 
 4   type          6808 non-null   object 
 5   count         6808 non-null   int64  
 6   amount        6808 non-null   float64
 7   timestamp     6808 non-null   object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 399.0+ KB


In [9]:
import mysql.connector
from tabulate import tabulate
import numpy as np

In [10]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'balaji',
    database = 'pulse'
)

mycursor = db.cursor(buffered=True)
mycursor

In [11]:
mycursor.execute("""create table if not exists topdatatransdistrict (
                topDataTransDistrictId int auto_increment primary key, state varchar(255), 
                year int(10), quarter varchar(255), district_name varchar(255), type varchar(10),
                count int(255), amount float, timestamp datetime)""")

In [12]:
sql = ("""insert into topdatatransdistrict (topDataTransDistrictId, state,year, quarter, 
       district_name, type, count, amount, timestamp) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update
       state = values(state), year = values(year), quarter = values(quarter), 
       district_name = values(district_name), type = values(type), count = values(count), 
       amount = values(amount), timestamp = values(timestamp)""")

for i in TopDistrictName.to_records().tolist():
    mycursor.execute(sql, i)

In [13]:
db.commit()

In [53]:
def DistrictTransactions(QUARTER, YEAR): 
    mycursor.execute("""select 
        quarter,
        year,
        district_name,
        sum(count) as Transactions_Count
        from topdatatransdistrict
        group by quarter, year, district_name""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'year', 'District_Name', 'Transactions_Count']

    def convert_lakh_crore(value):
        if value >= 10000000:
            return f'{value / 10000000 : .2f} Cr'
        else:
            return f'{value / 100000 : .2f} L'
        
    DistrictTransactions = pd.DataFrame(data=data, columns=columns)
    DistrictTransactions = DistrictTransactions[(DistrictTransactions['quarter']== QUARTER) & (DistrictTransactions['year'] == YEAR)]
    DistrictTransactions['Rank'] = DistrictTransactions['Transactions_Count'].rank(ascending=False)
    DistrictTransactions = DistrictTransactions.sort_values(by=['Transactions_Count', 'Rank'], ascending=[False, True])
    DistrictTransactions = DistrictTransactions.reset_index(drop=True)
    DistrictTransactions['Transactions_Count'] = DistrictTransactions['Transactions_Count'].apply(convert_lakh_crore)
    DistrictTransactions.drop(columns=['Rank'], inplace=True)
    TopDistrict = DistrictTransactions.head(10)
    return TopDistrict


In [54]:
DistrictTransactions('Q3', 2023)

,quarter,year,District_Name,Transactions_Count
0,Q3,2023,bengaluru urban,129.81 Cr
1,Q3,2023,pune,62.54 Cr
2,Q3,2023,rangareddy,43.94 Cr
3,Q3,2023,hyderabad,37.18 Cr
4,Q3,2023,jaipur,34.24 Cr
5,Q3,2023,medchal malkajgiri,34.16 Cr
6,Q3,2023,visakhapatnam,20.99 Cr
7,Q3,2023,gurugram,18.90 Cr
8,Q3,2023,gautam buddha nagar,18.05 Cr
9,Q3,2023,thane,17.04 Cr


# Top transaction Data Pincode

In [14]:
def Top_Transcations_Pincode():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})

    lis3 = []
    for l in lis:
        for m in l['data']['data']['pincodes']:
            amount_value = m['metric']['amount']
            amount_formatted = f'{amount_value : .2f}'

            data1 = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['entityName'],
                        type = m['metric']['type'],
                        count = m['metric']['count'],
                        amount = amount_formatted,
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data1)

    return lis3

In [15]:
TopPincodeName = pd.DataFrame(Top_Transcations_Pincode())
TopPincodeName['year'] = TopPincodeName['year'].astype(int)
# replace nan with 0 and inf with a suitable value(eg -1)
TopPincodeName['pincodes'] = TopPincodeName['pincodes'].fillna(0).replace([np.inf, -np.inf], -1).astype(float).astype(int)
TopPincodeName['amount'] = TopPincodeName['amount'].astype(float)
TopPincodeName['timestamp'] = pd.to_datetime(TopPincodeName['timestamp'], unit='ms')
TopPincodeName['timestamp'] = TopPincodeName['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopPincodeName

,state,year,quarter,pincodes,type,count,amount,timestamp
0,andaman-&-nicobar-islands,2018,Q1,744101,TOTAL,1622,2.769298e+06,2021-09-01 13:04:46
1,andaman-&-nicobar-islands,2018,Q1,744103,TOTAL,1223,2.238042e+06,2021-09-01 13:04:46
2,andaman-&-nicobar-islands,2018,Q1,744102,TOTAL,969,3.519060e+06,2021-09-01 13:04:46
3,andaman-&-nicobar-islands,2018,Q1,744105,TOTAL,685,1.298561e+06,2021-09-01 13:04:46
4,andaman-&-nicobar-islands,2018,Q1,744104,TOTAL,340,1.039715e+06,2021-09-01 13:04:46
...,...,...,...,...,...,...,...,...
8205,west-bengal,2023,Q3,700001,TOTAL,3536928,7.193522e+09,2023-10-18 06:54:33
8206,west-bengal,2023,Q3,721301,TOTAL,3464729,4.871566e+09,2023-10-18 06:54:33
8207,west-bengal,2023,Q3,700039,TOTAL,3405551,5.095825e+09,2023-10-18 06:54:33
8208,west-bengal,2023,Q3,732125,TOTAL,3347165,5.353831e+09,2023-10-18 06:54:33


In [16]:
mycursor.execute("""create table if not exists topdatatranspincode (
                 topDataTransPincodeId int auto_increment primary key, state varchar(255), 
                 year int(10), quarter varchar(10), pincodes int(10), type varchar(10), 
                 count bigint, amount float, timestamp datetime
                 )""")

In [17]:
sql = ("""insert into topdatatranspincode (topDataTransPincodeId, state, year, quarter, pincodes, type,
       count, amount, timestamp) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update 
       state = values(state), year = values(year), quarter = values(quarter), 
       pincodes = values(pincodes), type = values(type), count = values(count), amount = values(amount),
       timestamp = values(timestamp)""")

for i in TopPincodeName.to_records().tolist():
    mycursor.execute(sql, i)

In [18]:
db.commit()

In [83]:
def PincodeTransactions(QUARTER, YEAR):
    mycursor.execute("""select
        quarter,
        year,
        pincodes,
        count as Transactions_Count
        from topdatatranspincode
        group by year, quarter, pincodes""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'year', 'Pincodes', 'Transactions_Count']

    def convert_lakh_crore(value):
        if value >= 10000000:
            return f'{value / 10000000 : .2f} Cr'
        else:
            return f'{value / 100000 : .2f} L'

    PincodeTransactions = pd.DataFrame(data=data, columns=columns)
    # PincodeTransactions
    PincodeTransactions = PincodeTransactions[(PincodeTransactions['quarter'] == QUARTER) & (PincodeTransactions['year'] == YEAR)]
    PincodeTransactions['Rank'] = PincodeTransactions['Transactions_Count'].rank(ascending=False)
    PincodeTransactions = PincodeTransactions.sort_values(by=['Transactions_Count', 'Rank'], ascending=[False,True])
    PincodeTransactions = PincodeTransactions.reset_index(drop=True)
    PincodeTransactions['Transactions_Count'] = PincodeTransactions['Transactions_Count'].apply(convert_lakh_crore)
    PincodeTransactions.drop(columns=['Rank'], inplace=True)
    TopPincode = PincodeTransactions.head(10)
    return TopPincode

In [84]:
PincodeTransactions('Q1', 2018)

,quarter,year,Pincodes,Transactions_Count
0,Q1,2018,560001,29.17 L
1,Q1,2018,110006,26.36 L
2,Q1,2018,400008,22.28 L
3,Q1,2018,600003,20.10 L
4,Q1,2018,731101,17.65 L
5,Q1,2018,700009,13.91 L
6,Q1,2018,411001,10.56 L
7,Q1,2018,500012,9.33 L
8,Q1,2018,462001,8.33 L
9,Q1,2018,201301,6.90 L


# Top Users Data District

In [19]:
def Top_Users_District():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        district_name = k['name'],
                        registeredUsers = k['registeredUsers'],
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)

    return lis2

In [20]:
TopUsersDistrict = pd.DataFrame(Top_Users_District())
TopUsersDistrict['year'] = TopUsersDistrict['year'].astype(int)
TopUsersDistrict['timestamp'] = pd.to_datetime(TopUsersDistrict['timestamp'], unit='ms')
TopUsersDistrict['timestamp'] = TopUsersDistrict['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopUsersDistrict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6808 entries, 0 to 6807
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state            6808 non-null   object
 1   year             6808 non-null   int32 
 2   quarter          6808 non-null   object
 3   district_name    6808 non-null   object
 4   registeredUsers  6808 non-null   int64 
 5   timestamp        6808 non-null   object
dtypes: int32(1), int64(1), object(4)
memory usage: 292.7+ KB


In [21]:
mycursor.execute("""create table if not exists topdatausersdistrict (
                 topDataUsersDistrictId int auto_increment primary key, state varchar(255), 
                 year int(10), quarter varchar(10), district_name varchar(255), 
                 registeredUsers bigint, timestamp datetime )""")

In [22]:
sql = ("""insert into topdatausersdistrict (topDataUsersDistrictId, state, year, quarter, 
       district_name, registeredUsers, timestamp) values(%s, %s, %s, %s, %s, %s, %s)
       on duplicate key update 
       state = values(state), year = values(year), quarter = values(quarter), 
       district_name = values(district_name), registeredUsers = values(registeredUsers),
       timestamp = values(timestamp)
       """)

for i in TopUsersDistrict.to_records().tolist():
    mycursor.execute(sql,i)

In [23]:
db.commit()

In [95]:
def TopDistrictUsers(QUARTER, YEAR):
    mycursor.execute("""select 
        quarter,
        year,
        district_name,
        registeredUsers
        from topdatausersdistrict
        group by quarter, year, district_name""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'year', 'District_Name', 'RegisteredUsers']

    def convert_lakh_crore(value):
        if value >= 10000000:
            return f'{value/10000000 : .2f} Cr'
        else:
            return f'{value/100000 : .2f} L'
        
    DistrictUsers = pd.DataFrame(data=data, columns=columns)
    DistrictUsers = DistrictUsers[(DistrictUsers['quarter'] == QUARTER) & (DistrictUsers['year']== YEAR)]
    DistrictUsers['Rank'] = DistrictUsers['RegisteredUsers'].rank(ascending= False)
    DistrictUsers = DistrictUsers.sort_values(by=['RegisteredUsers', 'Rank'], ascending=[False, True])
    DistrictUsers['RegisteredUsers'] = DistrictUsers['RegisteredUsers'].apply(convert_lakh_crore)
    DistrictUsers.drop(columns=['Rank'], inplace=True)
    TopDistrictUsers = DistrictUsers.head(10)
    return TopDistrictUsers


In [112]:
TopDistrictUsers('Q1', 2018)

,quarter,year,District_Name,RegisteredUsers
2713,Q1,2018,bengaluru urban,19.22 L
3472,Q1,2018,pune,12.12 L
5128,Q1,2018,jaipur,9.01 L
3473,Q1,2018,mumbai suburban,7.19 L
5588,Q1,2018,hyderabad,6.55 L
5589,Q1,2018,rangareddy,6.35 L
3474,Q1,2018,thane,6.32 L
1586,Q1,2018,ahmadabad,5.73 L
5590,Q1,2018,medchal malkajgiri,5.49 L
68,Q1,2018,visakhapatnam,4.97 L


# Top Users Data Pincode

In [24]:
def Top_Users_Pincode():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis3 = []
    for l in lis:
        for m in l['data']['data']['pincodes']:
            data = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['name'],
                        registeredUsers = m['registeredUsers'],
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data)

    return lis3

In [25]:
TopUsersPincode = pd.DataFrame(Top_Users_Pincode())
TopUsersPincode['year'] = TopUsersPincode['year'].astype(int)
TopUsersPincode['pincodes'] = TopUsersPincode['pincodes'].astype(int)
TopUsersPincode['timestamp'] = pd.to_datetime(TopUsersPincode['timestamp'], unit='ms')
TopUsersPincode['timestamp'] = TopUsersPincode['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopUsersPincode

,state,year,quarter,pincodes,registeredUsers,timestamp
0,andaman-&-nicobar-islands,2018,Q1,744103,1608,2021-09-01 13:04:54
1,andaman-&-nicobar-islands,2018,Q1,744101,1108,2021-09-01 13:04:54
2,andaman-&-nicobar-islands,2018,Q1,744105,1075,2021-09-01 13:04:54
3,andaman-&-nicobar-islands,2018,Q1,744102,1006,2021-09-01 13:04:54
4,andaman-&-nicobar-islands,2018,Q1,744104,272,2021-09-01 13:04:54
...,...,...,...,...,...,...
8206,west-bengal,2023,Q3,700015,122316,2023-10-18 06:55:49
8207,west-bengal,2023,Q3,742304,119245,2023-10-18 06:55:49
8208,west-bengal,2023,Q3,721101,118798,2023-10-18 06:55:49
8209,west-bengal,2023,Q3,700150,114987,2023-10-18 06:55:49


In [26]:
mycursor.execute("drop table topdatauserspincode")

In [27]:
mycursor.execute("""create table if not exists topdatauserspincode (
                 topDataUsersPincodeid int auto_increment primary key, state varchar(255),
                 year int(10), quarter varchar(10), pincodes int, registeredUsers bigint, 
                 timestamp datetime)""")

In [28]:
sql = ("""insert into topdatauserspincode (topDataUsersPincodeId, state, year, quarter, pincodes,
       registeredUsers, timestamp)
       values(%s, %s, %s, %s, %s, %s, %s) 
       on duplicate key update
       state = values(state), year = values(year), quarter = values(quarter), 
       pincodes = values(pincodes), registeredUsers = values(registeredUsers), 
       timestamp = values(timestamp)""")

for i in TopUsersPincode.to_records().tolist():
    mycursor.execute(sql,i)

In [29]:
db.commit()

In [110]:
def TopPincodeUsers(QUARTER, YEAR): 
    mycursor.execute("""select 
        quarter,
        year,
        Pincodes,
        RegisteredUsers
        from topdatauserspincode
        group by quarter, year, pincodes""")

    out = mycursor.fetchall()
    data = list(out)
    columns = ['quarter', 'year', 'Pincodes', 'RegisteredUsers']

    def convert_lakh_crore(value):
        if value >= 100000:
            return f'{value/100000 : .2f} L'
        else:
            return f'{value /1000 : .2f} K'

    TopPincodeUsers = pd.DataFrame(data=data, columns=columns)
    TopPincodeUsers = TopPincodeUsers[(TopPincodeUsers['year'] == YEAR) & (TopPincodeUsers['quarter'] == QUARTER)]
    TopPincodeUsers['Rank'] = TopPincodeUsers['RegisteredUsers'].rank(ascending=False)
    # TopPincodeUsers
    TopPincodeUsers = TopPincodeUsers.sort_values(by=['RegisteredUsers', 'Rank'], ascending=[False, True])
    TopPincodeUsers = TopPincodeUsers.reset_index(drop=True)
    TopPincodeUsers['RegisteredUsers'] = TopPincodeUsers['RegisteredUsers'].apply(convert_lakh_crore)
    TopPincodeUsers.drop(columns=['Rank'], inplace=True)
    TopPincodeUsers= TopPincodeUsers.head(10)
    return TopPincodeUsers



In [111]:
TopPincodeUsers('Q3', 2023)

,quarter,year,Pincodes,RegisteredUsers
0,Q3,2023,201301,9.16 L
1,Q3,2023,110059,7.72 L
2,Q3,2023,560068,7.05 L
3,Q3,2023,421302,6.89 L
4,Q3,2023,122001,6.10 L
5,Q3,2023,500072,6.10 L
6,Q3,2023,110092,5.98 L
7,Q3,2023,110086,5.41 L
8,Q3,2023,121004,5.30 L
9,Q3,2023,201009,5.10 L
